In [53]:
import src.bsky

bsky_client = src.bsky.init()

handle = "coilysiren.me"
text = "\n".join(await src.bsky.get_author_feed_texts(bsky_client, "coilysiren.me", 25))

2025-04-02 23:39:08 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text--coilysiren.me prefix=bsky.get-author-feed-text- suffix=coilysiren.me
cursor: 2025-03-28T14:43:31.185Z
2025-04-02 23:39:08 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text-2025-03-28T14:43:31.185Z-coilysiren.me prefix=bsky.get-author-feed-text-2025-03-28T14:43:31.185Z suffix=coilysiren.me
cursor: 2025-03-25T11:18:08.551Z
2025-04-02 23:39:08 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text-2025-03-25T11:18:08.551Z-coilysiren.me prefix=bsky.get-author-feed-text-2025-03-25T11:18:08.551Z suffix=coilysiren.me
cursor: 2025-03-22T15:51:30.34Z
2025-04-02 23:39:08 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text-2025-03-22T15:51:30.34Z-coilysiren.me prefix=bsky.get-author-feed-text-2025-03-22T15:51:30.34Z suffix=coilysiren.me
cursor: 2025-03-14T12:32:15.902Z
2025-04-02 23:39:08

In [2]:
import nltk
import nltk.corpus

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/kai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
import subprocess
import spacy

subprocess.run(["spacy", "download", "en_core_web_lg"])
nlp = spacy.load("en_core_web_lg")

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.8.0/en_core_web_lg-3.8.0-py3-none-any.whl (400.7 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [47]:
import yake
import nltk
import nltk.corpus
import spacy
import yaml
import pprint

def remove_substring_entries(keyword_list):
    # Sort by length of keyword (longest first), then by score (higher is better)
    keyword_list.sort(key=lambda x: (-x[0], -len(x[1])))

    filtered_keywords = []
    seen_words = set()

    for score, keyword in keyword_list:
        words = set(keyword.split())  # Split phrase into individual words
        if not any(word in seen_words for word in words):
            filtered_keywords.append((score, keyword))
            seen_words.update(words)  # Add words to seen set

    return filtered_keywords

with open("stopwords.yml", "r") as _file:
    custom_stopwords = yaml.load(_file, yaml.Loader)

yake_kw_extractor = yake.KeywordExtractor(
    lan="en",
    top=50,
    dedupLim=1,
    stopwords=set(nltk.corpus.stopwords.words("english") + list(nlp.Defaults.stop_words) + custom_stopwords)
)
keywords = yake_kw_extractor.extract_keywords(text.lower())

keywords = [
    (pair[0], pair[1]) if type(pair[0]) != str else (pair[1], pair[0])
    for pair in keywords
] # this pair comes on in a different order depending on your operating system

keywords = remove_substring_entries(keywords)

print(pprint.pprint(keywords))


[(0.06226594632005874, 'server'),
 (0.061990562601449806, 'dems'),
 (0.06125068920237876, 'machine learning'),
 (0.06092129394242671, 'stop'),
 (0.06064442032810387, 'transgender mice'),
 (0.05998185380269443, 'social'),
 (0.0595197027028411, 'trans girl website'),
 (0.05934486133227008, 'devops engineer'),
 (0.058274659127150255, 'mobile app'),
 (0.058180496490271606, 'nextjs'),
 (0.05796732805901702, 'code'),
 (0.05666084242192651, 'github.com'),
 (0.05556134271131322, 'coilysiren'),
 (0.05518109135534268, 'leetcode'),
 (0.05241001038254615, 'redis'),
 (0.05173381567154017, 'twitter'),
 (0.05048643532961024, 'work laptop'),
 (0.050215513373595644, 'country'),
 (0.048908601682541, 'hate'),
 (0.046409077420662186, 'job'),
 (0.04540637654603328, 'states'),
 (0.044819379586473126, 'suggestions'),
 (0.042693420864084085, 'public'),
 (0.041594098622339815, 'working'),
 (0.04133681437705453, 'problem'),
 (0.03576566560065, 'bluesky apps'),
 (0.0354691635625986, 'works'),
 (0.035028302314763

In [55]:
import spacy
import json
import pprint

nlp = spacy.load("en_core_web_lg")  # Load large model for better similarity

# Load emojis from JSON
with open("emojis.json", "r") as _file:
    emojis = json.loads(_file.read())

emoji_scores = []

# Precompute NLP embeddings for emoji descriptions
emoji_nlp = [
    nlp(emojis[emoji_index]["description"])
    for emoji_index in range(len(emojis))
]

for keyword in keywords:
    _keyword = nlp(keyword[1])  # Process keyword text
    keyword_scores = []

    for emoji_index in range(len(emojis)):
        _emoji = emoji_nlp[emoji_index]  # Process emoji description
        keyword_score = keyword[0]

        # **Exact match logic**: If the keyword matches the emoji description exactly
        if keyword[1].lower() == emojis[emoji_index]["description"].lower():
            emoji_score = 1.0  # Exact match gets max similarity
        else:
            emoji_score = _keyword.similarity(_emoji)  # Otherwise, use semantic similarity

        keyword_scores.append([
            _keyword, emoji_score, emojis[emoji_index]["emoji"], emojis[emoji_index]["description"]
        ])

    # Sort emoji matches by highest similarity
    keyword_scores.sort(key=lambda x: -x[1])
    
    # Append best match to emoji_scores
    emoji_scores.append(keyword_scores[0])

# Sort overall results by highest similarity
emoji_scores.sort(key=lambda x: -x[1])

# Print results
pprint.pprint(emoji_scores)

print(f"{handle} talks about...")

for emoji_score in emoji_scores:
    print(emoji_score[2], emoji_score[0])


/var/folders/37/h0w8431d5dqb2pjq5ws463q00000gn/T/ipykernel_1384/2524784109.py:31: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  emoji_score = _keyword.similarity(_emoji)  # Otherwise, use semantic similarity


[[work laptop, 0.8737037669212877, '💻', 'laptop'],
 [game, 0.8588475456114495, '🎮', 'video game'],
 [black, 0.8536479492993099, '⚫', 'black circle'],
 [stop, 0.8452153448480786, '🛑', 'stop sign'],
 [mobile app, 0.8229440987707605, '📱', 'mobile phone'],
 [love, 0.7793053496256116, '💌', 'love letter'],
 [world, 0.7714790030477227, '🗺️', 'world map'],
 [states, 0.739881039640825, '🇺🇸', 'flag: United States'],
 [trans girl website, 0.7209500023475277, '👧', 'girl'],
 [machine learning, 0.6786142029555245, '🎰', 'slot machine'],
 [data, 0.6405320669100674, '🔢', 'input numbers'],
 [job, 0.6342265489954826, '👨\u200d💼', 'man office worker'],
 [signal, 0.633443405414455, '🔣', 'input symbols'],
 [state, 0.6324717996393164, '🇺🇸', 'flag: United States'],
 [country, 0.6253653921115787, '🇹🇫', 'flag: French Southern Territories'],
 [working, 0.6105500971586605, '🏃\u200d➡️', 'person running facing right'],
 [hate, 0.6098208879777762, '😱', 'face screaming in fear'],
 [public, 0.596301335524919, '🏞️', 'na